In [1]:
import pandas as pd
import numpy as np

# 1.0 Cleaning Data
## 1.1 Splitting violation codes out
First I want to break out the violations into their own table so we can extract the violation codes

In [2]:
healthdf = pd.read_csv('/home/michael/Downloads/Food_Inspections.csv')

In [3]:
healthdf.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2144504,TAQUERIA EL RANCHITO,TAQUERIA EL RANCHITO,60184.0,Restaurant,Risk 1 (High),2829 N MILWAUKEE AVE,CHICAGO,IL,60618.0,02/02/2018,Canvass Re-Inspection,Pass,31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVIC...,41.932657,-87.713056,"(41.93265693660558, -87.71305610116421)"
1,2144489,EL TACONAZO JR.,EL TACONAZO JR.,2354772.0,Restaurant,Risk 1 (High),4256 W FULLERTON AVE,CHICAGO,IL,60639.0,02/02/2018,Canvass,Pass w/ Conditions,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,41.924508,-87.733980,"(41.92450769084945, -87.73398032565292)"
2,2144480,BRICK HOUSE BAR AND GRILL,BRICK HOUSE BAR AND GRILL,1969907.0,Restaurant,Risk 1 (High),3152-3154 W IRVING PARK RD,CHICAGO,IL,60618.0,02/02/2018,Canvass Re-Inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.954032,-87.707720,"(41.95403155441393, -87.70771986355506)"
3,2144477,THE VEGGIE GRILL,THE VEGGIE GRILL,2563666.0,NaN,Risk 1 (High),614 W DIVERSEY AVE,CHICAGO,IL,60614.0,02/02/2018,License,Not Ready,NaN,41.932917,-87.644572,"(41.93291687750861, -87.64457163885304)"
4,2144474,SMOQUE BBQ,SMOQUE BBQ,1694866.0,Restaurant,Risk 1 (High),3800 N PULASKI RD,CHICAGO,IL,60641.0,02/02/2018,Canvass,Pass w/ Conditions,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,41.950073,-87.727657,"(41.95007328750793, -87.72765652046023)"


In [4]:
healthdf.iloc[0]['Violations']

'31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVICE ARTICLES PROPERLY STORED: NO REUSE OF SINGLE SERVICE ARTICLES - Comments: OBSERVED CUPS USED TO DISPENSE FOOD IN THE CONTAINERS OF CHEESE AND GIARDINNARA IN THE COLD HOLDING REFRIGERATION UNIT. INSTRUCTED TO USE HANDLED SCOOPS TO DISPENSE FOOD AND TO STORE HANDLED SCOOPS EITHER IN THE FOOD WITH THE HANDLE FACING UP OUT OF THE FOOD OR IN A SEPARATE CONTAINER IN BETWEEN USE. | 35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTRUCTED PER CODE: GOOD REPAIR, SURFACES CLEAN AND DUST-LESS CLEANING METHODS - Comments: MUST DETAIL CLEAN THE FOOD SPLATTER ON THE WALLS BEHIND THE HOT HOLDING UNITS AND THE REFRIGERATION UNITS ACROSS THE COOKING EQUIPMENT ON THE LINE. \n | 36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF LIGHT PROVIDED, FIXTURES SHIELDED - Comments: OBSERVED MISSING LIGHT SHIELDS ABOVE THE PREP AREA IN THE KITCHEN. INSTRUCTED TO PROVIDE MISSING SHIELDS TO PROTECT LIGHT FIXTURES AT ALL TIMES.'

In [5]:
violations = healthdf[['Inspection ID','Violations']]
violations = violations.dropna()
violations.shape

(131495, 2)

In [6]:
def split_codes(df):
    '''
    function to split the violations into an inspection id, violation code, 
    violation code description and the comment on the violation
    '''
    result = pd.DataFrame()
    for index, rows in df.iterrows():
        x = pd.DataFrame(rows['Violations'].split('|'),columns = ['fulldesc'])
        x['fulldesc'] = x['fulldesc'].str.strip()
        x['Inspection ID'] = rows['Inspection ID']
        x['vcode'] = x['fulldesc'].str.split('.').str[0]
        x['codedesc'] = x['fulldesc'].str.split(' - ').str[0]
        x['comment'] = x['fulldesc'].str.split(' - ').str[1]
        x['comment'] = x['comment'].str[9:]
        result = result.append(x)
    return result[['Inspection ID','vcode','codedesc','comment']]

In [7]:
viodf = split_codes(violations)

In [267]:
viodf.head()

,Inspection ID,vcode,codedesc,comment,vioid
0,2144504,31,31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVIC...,OBSERVED CUPS USED TO DISPENSE FOOD IN THE CO...,1
1,2144504,35,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",MUST DETAIL CLEAN THE FOOD SPLATTER ON THE WA...,2
2,2144504,36,36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF...,OBSERVED MISSING LIGHT SHIELDS ABOVE THE PREP...,3
0,2144489,3,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,OBSERVED IMPROPER TEMPERATURES OF POTENTIALLY...,4
1,2144489,32,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,OBSERVED WORN CUTTING BOARD WITH DARK GROOVE ...,5


In [268]:
viodf.iloc[3]#['comment']

Inspection ID                                              2144489
vcode                                                            3
codedesc         3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...
comment           OBSERVED IMPROPER TEMPERATURES OF POTENTIALLY...
vioid                                                            4
Name: 0, dtype: object

In [226]:
viodf.head()

,Inspection ID,vcode,codedesc,comment
0,2144504,31,31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVIC...,OBSERVED CUPS USED TO DISPENSE FOOD IN THE CO...
1,2144504,35,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",MUST DETAIL CLEAN THE FOOD SPLATTER ON THE WA...
2,2144504,36,36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF...,OBSERVED MISSING LIGHT SHIELDS ABOVE THE PREP...
0,2144489,3,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,OBSERVED IMPROPER TEMPERATURES OF POTENTIALLY...
1,2144489,32,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,OBSERVED WORN CUTTING BOARD WITH DARK GROOVE ...


In [227]:
viodf['vioid'] = range(1, len(viodf) + 1)

In [228]:
viodf.to_pickle('/home/michael/chicagohealthinspections/data/violations.pkl')

## 1.2 Clean Yelp Fields
Now we need to fix some of the business info before we add it all to the database

In [12]:
businfodf = pd.read_pickle('/home/michael/chicagohealthinspections/data/businessinfo.pkl')

In [14]:
businfodf = businfodf.drop_duplicates(subset = 'id')
businfodf

,categories,coordinates.latitude,coordinates.longitude,display_phone,error.code,error.description,hours,id,image_url,is_claimed,...,location.state,location.zip_code,name,phone,photos,price,rating,review_count,transactions,url
0,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",41.932760,-87.712900,(773) 227-1688,NaN,NaN,"[{'open': [{'is_overnight': True, 'start': '00...",el-ranchito-restaurant-chicago-5,https://s3-media4.fl.yelpcdn.com/bphoto/yVbkID...,True,...,IL,60618,El Ranchito Restaurant,+17732271688,[https://s3-media4.fl.yelpcdn.com/bphoto/yVbkI...,$,2.0,320.0,"[delivery, pickup]",https://www.yelp.com/biz/el-ranchito-restauran...
0,"[{'alias': 'tacos', 'title': 'Tacos'}]",41.924748,-87.714682,(773) 278-8944,NaN,NaN,"[{'open': [{'is_overnight': True, 'start': '08...",el-taconazo-chicago-4,https://s3-media3.fl.yelpcdn.com/bphoto/p9_vHn...,False,...,IL,60639,El Taconazo,+17732788944,[https://s3-media3.fl.yelpcdn.com/bphoto/p9_vH...,$,3.5,36.0,"[delivery, pickup]",https://www.yelp.com/biz/el-taconazo-chicago-4...
0,"[{'alias': 'tradamerican', 'title': 'American ...",41.954235,-87.707764,(773) 588-5558,NaN,NaN,NaN,brick-house-cafe-chicago,https://s3-media1.fl.yelpcdn.com/bphoto/RkMUPl...,False,...,IL,60618,Brick House Cafe,+17735885558,[https://s3-media1.fl.yelpcdn.com/bphoto/RkMUP...,$,4.5,15.0,[],https://www.yelp.com/biz/brick-house-cafe-chic...
0,"[{'alias': 'bbq', 'title': 'Barbeque'}]",41.950210,-87.727950,(773) 545-7427,NaN,NaN,"[{'open': [{'is_overnight': False, 'start': '1...",smoque-bbq-chicago,https://s3-media4.fl.yelpcdn.com/bphoto/a_qNUn...,True,...,IL,60641,Smoque BBQ,+17735457427,[https://s3-media4.fl.yelpcdn.com/bphoto/a_qNU...,$$,4.5,3683.0,[],https://www.yelp.com/biz/smoque-bbq-chicago?ad...
0,"[{'alias': 'catering', 'title': 'Caterers'}, {...",41.984060,-87.774910,(773) 792-2262,NaN,NaN,"[{'open': [{'is_overnight': False, 'start': '0...",gala-banquets-chicago,https://s3-media1.fl.yelpcdn.com/bphoto/8_fV1a...,True,...,IL,60646,Gala Banquets,+17737922262,[https://s3-media1.fl.yelpcdn.com/bphoto/8_fV1...,NaN,5.0,17.0,[],https://www.yelp.com/biz/gala-banquets-chicago...
0,"[{'alias': 'grocery', 'title': 'Grocery'}]",41.997585,-87.691743,(773) 764-3737,NaN,NaN,NaN,mehrab-chicago,https://s3-media3.fl.yelpcdn.com/bphoto/HDrilh...,False,...,IL,60659,Mehrab,+17737643737,[https://s3-media3.fl.yelpcdn.com/bphoto/HDril...,$$,3.0,3.0,[],https://www.yelp.com/biz/mehrab-chicago?adjust...
0,"[{'alias': 'breakfast_brunch', 'title': 'Break...",41.959579,-87.708003,(773) 961-7023,NaN,NaN,"[{'open': [{'is_overnight': False, 'start': '0...",cafe-san-juan-chicago,https://s3-media3.fl.yelpcdn.com/bphoto/Vo0_9i...,True,...,IL,60618,Cafe San Juan,+17739617023,[https://s3-media3.fl.yelpcdn.com/bphoto/Vo0_9...,$$,4.5,44.0,"[pickup, delivery]",https://www.yelp.com/biz/cafe-san-juan-chicago...
0,"[{'alias': 'cuban', 'title': 'Cuban'}, {'alias...",41.924415,-87.742347,(773) 227-1020,NaN,NaN,"[{'open': [{'is_overnight': False, 'start': '0...",señor-pan-cafe-chicago-4,https://s3-media4.fl.yelpcdn.com/bphoto/ee5mRG...,True,...,IL,60639,Señor Pan Cafe,+17732271020,[https://s3-media4.fl.yelpcdn.com/bphoto/ee5mR...,$,4.0,422.0,"[delivery, pickup]",https://www.yelp.com/biz/se%C3%B1or-pan-cafe-c...
0,"[{'alias': 'mexican', 'title': 'Mexican'}]",41.908806,-87.726135,(773) 276-5825,NaN,NaN,"[{'open': [{'is_overnight': True, 'start': '08...",pozoleria-san-juan-chicago,https://s3-media1.fl.yelpcdn.com/bphoto/VbLUlu...,True,...,IL,60651,Pozoleria San Juan,+17732765825,[https://s3-media1.fl.yelpcdn.com/bphoto/VbLUl...,$,3.5,62.0,[],https://www.yelp.com/biz/pozoleria-san-juan-ch...
0,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",41.865590,-87.626230,(312) 427-5507,NaN,NaN,NaN,subway-chicago-44,https://s3-media1.fl.yelpcdn.com/bphoto/YvJFH6...,False,...,IL,60605,Subway,+13124275507,[https://s3-media1.fl.yelpcdn.com/bphoto/YvJFH...,$,3.0,9.0,"[delivery, pickup]",https://www.yelp.com/biz/subway-chicago-44?adj...


In [15]:
cat = businfodf[['id','categories']]
cat = cat.groupby('id').categories.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)
cat['catid'] = range(1, len(cat) + 1)
cat.head()

,id,alias,title,catid
0,1-chop-suey-chicago,chinese,Chinese,1
1,1000-liquors-chicago-2,beer_and_wine,"Beer, Wine & Spirits",2
2,1048-sky-lounge-wrigley-rooftop-chicago-4,stadiumsarenas,Stadiums & Arenas,3
3,1048-sky-lounge-wrigley-rooftop-chicago-4,venues,Venues & Event Spaces,4
4,10pin-bowling-lounge-chicago,bowling,Bowling,5


In [16]:
cat.to_pickle('/home/michael/chicagohealthinspections/data/categories.pkl')

In [216]:
cat.groupby('title')[['catid']].count().sort_values('catid',ascending=False).head(10)

,catid
title,
Mexican,1142
Sandwiches,1044
Coffee & Tea,969
Fast Food,883
Pizza,825
American (Traditional),743
Breakfast & Brunch,617
Grocery,607
American (New),606


In [218]:
businfodf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13233 entries, 0 to 0
Data columns (total 28 columns):
categories                  13232 non-null object
coordinates.latitude        13232 non-null float64
coordinates.longitude       13232 non-null float64
display_phone               13232 non-null object
error.code                  1 non-null object
error.description           1 non-null object
hours                       9516 non-null object
id                          13232 non-null object
image_url                   13232 non-null object
is_claimed                  13232 non-null object
is_closed                   13232 non-null object
location.address1           13232 non-null object
location.address2           11297 non-null object
location.address3           11943 non-null object
location.city               13232 non-null object
location.country            13232 non-null object
location.cross_streets      13232 non-null object
location.display_address    13232 non-null object
lo

In [248]:
businesses = businfodf[['id','name','price','rating','review_count']].copy(deep=True)
businesses['BusId'] = range(1, len(businesses) + 1)
businesses = businesses.dropna()
businesses = businesses.replace('Sink | Swim','Sink or Swim')
businesses.head()

,id,name,price,rating,review_count,BusId
0,el-ranchito-restaurant-chicago-5,El Ranchito Restaurant,$,2.0,320.0,1
0,el-taconazo-chicago-4,El Taconazo,$,3.5,36.0,2
0,brick-house-cafe-chicago,Brick House Cafe,$,4.5,15.0,3
0,smoque-bbq-chicago,Smoque BBQ,$$,4.5,3683.0,4
0,mehrab-chicago,Mehrab,$$,3.0,3.0,6


In [250]:
businesses.iloc[10348]

id              four-sisters-chicago
name                    Four Sisters
price                              $
rating                           4.5
review_count                       4
BusId                          12603
Name: 0, dtype: object

In [251]:
businesses.to_pickle('/home/michael/chicagohealthinspections/data/businesses.pkl')

## 1.3 Updating Inspection Data
I need to assign location ids as well as get a count for the number of inspections per location

In [20]:
idlink = pd.read_pickle('/home/michael/chicagohealthinspections/data/idlink.pkl')

In [257]:
healthdf = pd.merge(healthdf, idlink, how='inner')
inspectdata = healthdf[['Inspection ID','License #','Facility Type',
                        'Risk','Inspection Date','Inspection Type',
                        'Results','LocId']].copy(deep=True)
inspectdata['Inspection Date'] = pd.to_datetime(inspectdata['Inspection Date'])
reslist = ['Fail','Pass','Pass w/ Conditions']
inspectdata = inspectdata[inspectdata['Results'].isin(reslist)]
inspectdata = inspectdata.sort_values(['LocId','Inspection Date'],ascending = [True,True])
inspectdata['inspcount'] = inspectdata.groupby('LocId').cumcount() + 1
inspectdata['License #'] = inspectdata['License #'].fillna(0) 

In [259]:
inspectdata.iloc[38314]

Inspection ID                  1188285
License #                            0
Facility Type            Special Event
Risk                   Risk 2 (Medium)
Inspection Date    2012-07-25 00:00:00
Inspection Type                Canvass
Results                           Pass
LocId                             6511
inspcount                            2
Name: 41042, dtype: object

In [260]:
inspectdata.to_pickle('/home/michael/chicagohealthinspections/data/inspectdata.pkl')

In [212]:
x = inspectdata.groupby('Results')[['LocId']].count()
x.columns = ['Inspections']
x.head()

,Inspections
Results,
Fail,20601
Pass,66392
Pass w/ Conditions,11892


In [215]:
fails = inspectdata[inspectdata['Results']=='Fail']
x = fails.groupby('Inspection Type',as_index=False)['LocId'].count().sort_values('LocId',ascending =False).head(5)
x.columns = ['Inspection Type','Failed Inspections']
x.head()

,Inspection Type,Failed Inspections
1,Canvass,10263
10,License,3999
3,Complaint,3648
2,Canvass Re-Inspection,563
28,Short Form Complaint,563


In [159]:
complaint = inspectdata[inspectdata['Inspection Type']=='Complaint']
complaint.groupby('Results',as_index=False)['LocId'].count().sort_values('LocId',ascending =False).head(5)

,Results,LocId
1,Pass,5614
0,Fail,3648
2,Pass w/ Conditions,2075


In [200]:
idlink[idlink['id'] == 'coles-chicago']

,DBA Name,Address,Latitude,Longitude,LocId,id


In [201]:
inspectdata[inspectdata['LocId']==8020]

,Inspection ID,License #,Facility Type,Risk,Inspection Date,Inspection Type,Results,LocId,inspcount
51926,134326,1979171.0,Restaurant,Risk 1 (High),2010-03-04,License,Fail,8020,1
51927,134325,1979165.0,Restaurant,Risk 1 (High),2010-03-04,License-Task Force,Fail,8020,2
51924,134349,1979171.0,Restaurant,Risk 1 (High),2010-03-17,Task Force Liquor 1475,Pass,8020,3
51925,134348,1979165.0,Restaurant,Risk 1 (High),2010-03-17,License Re-Inspection,Pass,8020,4
51923,660065,1979165.0,Restaurant,Risk 1 (High),2012-01-19,Suspected Food Poisoning,Fail,8020,5
51922,626385,1979165.0,Restaurant,Risk 1 (High),2012-01-26,Suspected Food Poisoning,Pass,8020,6
51921,1335847,1979165.0,Restaurant,Risk 1 (High),2013-05-31,Canvass,Fail,8020,7
51920,1336005,1979165.0,Restaurant,Risk 1 (High),2013-06-10,Canvass Re-Inspection,Pass,8020,8
51919,1494120,1979165.0,Restaurant,Risk 1 (High),2014-08-28,Canvass,Pass,8020,9
51918,1445350,1979165.0,Restaurant,Risk 1 (High),2015-09-25,Canvass,Pass w/ Conditions,8020,10


In [199]:
viodf[viodf['Inspection ID']==1632498]#.iloc[0]['comment']

,Inspection ID,vcode,codedesc,comment
0,1632498,16,"16. FOOD PROTECTED DURING STORAGE, PREPARATION...",VIOLATION CORRECTED \nPRESENTLY ICE MACHINE C...
1,1632498,32,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,NO SPLASH GUARD IS PROVIDED AT EXPOSED HAND S...
2,1632498,33,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,VIOLATION CORRECTED
3,1632498,35,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",VIOLATION CORRECTED
4,1632498,39,39. LINEN: CLEAN AND SOILED PROPERLY STORED,VIOLATION CORRECTED
5,1632498,40,40. REFRIGERATION AND METAL STEM THERMOMETERS ...,REPLACE BROKEN THERMOMETER INSIDE THE COCA-CO...
6,1632498,36,36. LIGHTING: REQUIRED MINIMUM FOOT-CANDLES OF...,BURNED LIGHT BULB INSIDE THE COCA-COLA REACH-...
